## General Description

This script reads data from a VTK EnSight Gold binary file and processes it to extract information about solar shading coefficients and the geometry of cells over a series of timesteps. The key steps include:

1. **Setting up the Reader**: The script initializes a VTK reader for the EnSight Gold binary file format and configures it to read all variables from a specified case file.

2. **Processing Data for Each Timestep**: For each timestep, the script updates the reader with the current time value, retrieves cell data including solar shading coefficients, and extracts the geometric points defining each cell. This data is combined and  converted into a pandas DataFrame with appropriate column names.

3. **Calculating the Area of Triangles**: A function calculates the area of a triangle given its vertices. This function is used to compute the area for each triangle (cell) in the DataFrame, and these areas are added as a new column in the DataFrame.

4. **Saving the Data**: The resulting DataFrame is saved to a CSV file for further analysis or use.

This script provides a comprehensive framework for reading, processing, and analyzing time-dependent geometric and scalar data from a VTK EnSight Gold binary file.


In [1]:
import vtk

# Setup the reader
# We suppose that the necessary files are in the same directory as the notebook
case_file = "strasbourg_sm_lod1/strasbourg_sm_lod1/City_Energy_Modeling.case"
# Setup the reader
reader = vtk.vtkEnSightGoldBinaryReader()
reader.SetCaseFileName(case_file)
reader.ReadAllVariablesOn()  # Ensure all variables are read
reader.Update()

# Retrieve the output
output = reader.GetOutput()
timeset=reader.GetTimeSets()
time=timeset.GetItem(0)
timesteps=time.GetSize()

In [2]:
print("Number of timesteps: ", timesteps)
print("Number of cells: ", reader.GetOutput().GetBlock(0).GetNumberOfCells())
print("Number of scalar points?: ", reader.GetOutput().GetBlock(0).GetCellData().GetArray("solar_shading_coeff").GetNumberOfTuples())

Number of timesteps:  731
Number of cells:  212692
Number of scalar points?:  212692


In [3]:
print(output.GetNumberOfCells())

212692


## Panda Dataframes

In [15]:
import numpy as np
import pandas as pd

# Initialize an empty list to accumulate the arrays
data = []

for t in range(timesteps):
    reader.SetTimeValue(time.GetValue(t))
    reader.Update()
    output = reader.GetOutput().GetBlock(0)
    cell_data = output.GetCellData()
    sc = cell_data.GetArray("solar_shading_coeff")


    for i in range(output.GetNumberOfCells()):
        cell = output.GetCell(i)
        pts = cell.GetPoints()
        scalar = sc.GetTuple(i)
        np_pts = np.array([pts.GetPoint(j) for j in range(pts.GetNumberOfPoints())])
        
        flattened_pts = np_pts.flatten()
        combined_data = np.append(flattened_pts, scalar[0])
        
        # Append the timestep and combined data to the list
        data.append([t] + combined_data.tolist())
        # data.append(np.insert(combined_data, 0, t))

# Create column names for the DataFrame
columns = ['timestep'] + ['x_1'] + ['y_1'] + ['z_1'] + ['x_2'] + ['y_2'] + ['z_2'] + ['x_3'] + ['y_3'] + ['z_3'] + ['scalar']

# Convert the list to a DataFrame
df = pd.DataFrame(data, columns=columns)

# Function to calculate the area of a triangle given its vertices
def calculate_triangle_area(p1, p2, p3):
    # Calculate the vectors for two sides of the triangle
    v1 = p2 - p1
    v2 = p3 - p1
    # Calculate the cross product of the vectors
    cross_product = np.cross(v1, v2)
    # Calculate the area of the triangle (0.5 * magnitude of the cross product)
    area = 0.5 * np.linalg.norm(cross_product)
    return area

# Calculate the area for each triangle and add it as a new column
areas = []
for index, row in df.iterrows():
    p1 = np.array([row['x_1'], row['y_1'], row['z_1']])
    p2 = np.array([row['x_2'], row['y_2'], row['z_2']])
    p3 = np.array([row['x_3'], row['y_3'], row['z_3']])
    area = calculate_triangle_area(p1, p2, p3)
    areas.append(area)

df['area'] = areas

# Print the resulting DataFrame
print(df.head())

   timestep         x_1         y_1  z_1         x_2         y_2  z_2  \
0         0 -539.268982  600.049988  0.0 -538.495972  599.625000  0.0   
1         0 -541.656982  611.940979  0.0 -527.434998  605.570984  0.0   
2         0 -527.434998  605.570984  0.0 -538.495972  599.625000  0.0   
3         0 -487.437988  635.723999  0.0 -490.282013  625.955994  0.0   
4         0 -488.596985  642.520020  0.0 -487.437988  635.723999  0.0   

          x_3         y_3  z_3  scalar        area  
0 -538.987976  599.625000  0.0     0.0    0.104548  
1 -539.268982  600.049988  0.0     0.0   76.950973  
2 -539.268982  600.049988  0.0     0.0    4.648543  
3 -518.830994  625.955994  0.0     0.0  139.433298  
4 -518.830994  625.955994  0.0     0.0  112.334297  


In [11]:
print(df.tail())

         timestep     point_1      point_2  point_3     point_4      point_5  \
5104603        23  725.992004 -1084.459961      0.0  725.992004 -1084.459961   
5104604        23  739.231018 -1185.939941      0.0  737.825989 -1185.089966   
5104605        23  737.825989 -1185.089966      0.0  737.825989 -1185.089966   
5104606        23  763.109985 -1150.270020      0.0  760.265015 -1155.369995   
5104607        23  760.265015 -1155.369995      0.0  760.265015 -1155.369995   

         point_6     point_7      point_8  point_9  scalar       area  
5104603      5.2  726.484009 -1079.369995      5.2     1.0  13.295592  
5104604      0.0  739.231018 -1185.939941      6.5     1.0   5.336898  
5104605      6.5  739.231018 -1185.939941      6.5     1.0   5.336898  
5104606      0.0  763.109985 -1150.270020      3.0     1.0   8.759744  
5104607      3.0  763.109985 -1150.270020      3.0     1.0   8.759744  


In [13]:
df.to_csv('initial_dataframe.csv', index=False)